# 第3节：校准与基准稳态 

## 表 3.1--模型核心参数校准

In [5]:
% =========================================================================
% == SCRIPT: Tab3_1.m
% == 版本: [v2.7 - 精简换行版]
% ==
% == 核心修改:
% ==   - [!!! 格式优化 !!!] 采纳您的建议，在 \midrule, \toprule,
% ==     \bottomrule 和 \multicolumn 这些本身就定义了行结构的命令后，
% ==     不再输出多余的换行符 "\\"。
% =========================================================================
clear; clc;
addpath(pwd);
fprintf('--- 启动 LaTeX 参数表直接生成脚本 (v2.7) ---\n');

%% --- 1. 加载数据并定义输出文件名 ---
data_filename = 'SS/data_for_transition.mat';
output_filename = 'tex/tab/Tab3.1.tex';

try
    load(data_filename, 'data_for_transition');
    cS = data_for_transition.cS;
    fprintf('   ✅ 已从 "%s" 加载参数结构体 cS。\n', data_filename);
catch ME
    error('无法加载转轨数据文件 "%s"。请先成功运行 main_run_ss.m。\n错误信息: %s', data_filename, ME.message);
end

%% --- 2. 构建参数数据 (Cell Array) ---
% 格式: { '命令或参数名', '符号', '数值', '来源', is_full_line_command }
% 最后一列是布尔值，用于标记是否为独占一行的命令
params_data = {
    '\midrule[0.75pt]', '', '', '', true; 
    '\multicolumn{4}{l}{\textbf{A. 偏好与生命周期参数}}\\', '', '', '', true;
    '相对风险规避系数', '$\sigma$', sprintf('%.1f', cS.sigma), '标准宏观文献', false;
    '主观贴现因子 (年化)', '$\beta_{annual}$', sprintf('%.3f', cS.beta^(1/cS.time_Step)), '校准目标', false;
    '人口稳态增长率 (年化)', '$n_{ss}$', sprintf('%.1f%%', cS.n_ss * 100), '长期人口预测', false;
    '存活率', '$\phi_a$', '时变', 'UN WPP (2024)', false;
    '模型期长度', '', sprintf('%d年', cS.time_Step), '模型设定', false;
    '退休年龄', '$a_{R}$', sprintf('%d岁', cS.ageRetire_orig), '模型设定', false;
    '\midrule', '', '', '', true; 
    '\multicolumn{4}{l}{\textbf{B. 生产技术参数}}\\', '', '', '', true;
    '技术进步率 (年化)', '$g_{A,ss}$', sprintf('%.1f%%', cS.g_A_ss * 100), '长期增长假设', false;
    '私人资本产出弹性', '$\alpha$', sprintf('%.3f', cS.alpha), 'Penn World Table', false;
    '公共资本产出弹性', '$\gamma$', sprintf('%.3f', cS.gamma), '校准目标', false;
    '私人资本折旧率 (年化)', '$\delta_k$', sprintf('%.1f%%', (1-(1-cS.ddk)^(1/cS.time_Step))*100), 'NIPA数据', false;
    '公共资本折旧率 (年化)', '$\delta_g$', sprintf('%.1f%%', (1-(1-cS.ddk_g)^(1/cS.time_Step))*100), '校准目标', false;
    '\midrule', '', '', '', true;
    '\multicolumn{4}{l}{\textbf{C. 财政与PAYG养老金}}\\', '', '', '', true;
    '资本所得税率', '$\tau_k$', sprintf('%.1f%%', cS.tau_k * 100), '中国现行税制', false;
    '劳动所得税率', '$\tau_l$', sprintf('%.1f%%', cS.tau_l * 100), '中国现行税制', false;
    '消费税率', '$\tau_c$', sprintf('%.1f%%', cS.tau_c * 100), '中国现行税制', false;
    '政府消费 / GDP', '$G_c/Y$', sprintf('%.1f%%', cS.G_c_to_Y_ratio_ss * 100), '财政数据', false;
    '公共投资 / GDP', '$I_g/Y$', sprintf('%.1f%%', cS.I_g_to_Y_ratio_ss * 100), '财政数据', false;
    'PAYG缴费率', '$\theta_t$', '时变路径', '内生以平衡PAYG', false;
    '\midrule', '', '', '', true;
    '\multicolumn{4}{l}{\textbf{D. 个人养老金 (PPS) 制度}}\\', '', '', '', true;
    'PPS最大缴费率 (占劳动收入)', '$pps_{max}$', sprintf('%.1f%%', cS.pps_max * 100), '中国个人养老金政策', false;
    'PPS提取率 (占PPS资产)', '', sprintf('%.1f%%', cS.pps_withdrawal_rate * 100), '模型简化假设', false;
    'PPS提取税率', '$\tau_{pps}$', sprintf('%.1f%%', cS.pps_tax_rate_withdrawal * 100), '中国个人养老金政策', false;
};
fprintf('   ✅ 已构建参数数据。\n');

%% --- 3. 准备输出目录并打开文件 ---
[output_dir, ~, ~] = fileparts(output_filename);
if ~exist(output_dir, 'dir')
    mkdir(output_dir);
    fprintf('   已创建新目录: "%s"\n', output_dir);
end

fileID = fopen(output_filename, 'w', 'n', 'UTF-8');
if fileID == -1, error('无法创建或打开最终输出文件: %s', output_filename); end
fprintf('   正在将完整的LaTeX表格代码写入 "%s"...\n', output_filename);

% --- 写入表格外层框架 ---
fprintf(fileID, '\\begin{table}[htbp]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{模型核心参数校准}\n');
fprintf(fileID, '\\label{tab:param_calibration}\n');
fprintf(fileID, '\\begin{threeparttable}\n');
fprintf(fileID, '\\begin{tabular}{lccc}\n');
fprintf(fileID, '\\toprule\n');
fprintf(fileID, '参数名 & 参数符号 & 参数值 & 参数来源 \\\\\n');

% --- 循环写入表格主体内容 ---
for i = 1:size(params_data, 1)
    % --- 统一进行转义处理 ---
    col1 = strrep(params_data{i, 1}, '%', '\%');
    col2 = strrep(params_data{i, 2}, '%', '\%');
    col3 = strrep(params_data{i, 3}, '%', '\%');
    col4 = strrep(params_data{i, 4}, '%', '\%');
    is_command = params_data{i, 5};
    
    if is_command
        % 如果是整行命令，直接打印，不加换行符 "\\"
        fprintf(fileID, '%s\n', col1);
    else
        % 如果是普通的数据行，用 "&" 连接并换行
        fprintf(fileID, '%s & %s & %s & %s \\\\\n', col1, col2, col3, col4);
    end
end

% --- 写入表格结尾和注释 ---
fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}\n'); 
fprintf(fileID, '\\begin{tablenotes}[para]\n');
fprintf(fileID, '  \\item[注] 主观贴现因子、折旧率和人口增长率已根据模型周期长度 (%d年) 进行年化处理以便解读。\n', cS.time_Step);
fprintf(fileID, '  PAYG缴费率($\\theta_t$)和存活率($\\phi_a$)为时变路径，详情见正文。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}');

% --- 关闭文件 ---
fclose(fileID);

fprintf('   ✅ 成功生成语法正确的LaTeX表格文件: %s\n', output_filename);
fprintf('\n--- 脚本执行完毕 ---\n');

--- 启动 LaTeX 参数表生成脚本 ---
   ✅ 已从 "SS/data_for_transition.mat" 加载参数结构体 cS。
   ✅ 已根据模型设定构建参数表。
   ✅ 已调用 table2latex 生成临时文件 "temp_table_for_script.tex"。
   正在将表格内容写入最终框架并保存至 "Tab3.1.txt"...
   ✅ 成功生成最终的LaTeX表格文件: Tab3.1.txt

--- 脚本执行完毕 ---


## 表 3.2--稳态宏观经济结果比较

In [ ]:
% =========================================================================
% == SCRIPT: ss_comparison.m
% == 版本: [v1.0 - 稳态结果LaTeX表格生成器]
% ==
% == 目的:
% ==   1. 加载"无PPS" (ss_nopps) 和 "有PPS" (ss_pps) 的稳态结果。
% ==   2. 调用国民经济核算函数生成报告。
% ==   3. 选取核心宏观变量，比较两个稳态的差异。
% ==   4. 自动生成一个精美的、可直接用于论文的LaTeX表格 (Tab3.2.tex)。
% ==
% == 前置条件:
% ==   - 已成功运行 main_run_ss.m 并生成以下两个文件:
% ==     - SS/data_for_transition_nopps.mat
% ==     - SS/data_for_transition_pps.mat
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== 稳态结果比较与LaTeX表格生成脚本 (v1.0) ===\n\n');

%% --- 1. 加载数据 ---
fprintf('--- 1. 加载两个情景的稳态数据 ---\n');

% --- 文件路径定义 ---
file_path_nopps = 'SS/data_for_transition_nopps.mat';
file_path_pps = 'SS/data_for_transition_pps.mat';
output_tex_filename = 'tex/tab/Tab3.2_ss_comparison.tex';

% --- 加载 "无PPS" (基准) 数据 ---
try
    data_nopps = load(file_path_nopps, 'data_for_transition');
    ss_nopps = data_nopps.data_for_transition.ss0; % 初始稳态
    cS_nopps = data_nopps.data_for_transition.cS;
    paramS_nopps = data_nopps.data_for_transition.paramS0;
    % 从dist0中恢复Z_norm
    dist0_abs_nopps = data_nopps.data_for_transition.dist0;
    Z_raw_nopps = sum(dist0_abs_nopps, [1 2 3]);
    Z_norm_nopps = Z_raw_nopps ./ sum(Z_raw_nopps);
    fprintf('   ✅ 已加载 "无PPS" (ss_nopps) 稳态数据。\n');
catch ME
    error('加载无PPS数据失败: %s', ME.message);
end

% --- 加载 "有PPS" (改革) 数据 ---
try
    data_pps = load(file_path_pps, 'data_for_transition');
    ss_pps = data_pps.data_for_transition.ssF; % 终期稳态
    cS_pps = data_pps.data_for_transition.cS;
    paramS_pps = data_pps.data_for_transition.paramSF;
    % 终期稳态使用其自身的Z分布
    Z_norm_pps = cS_pps.Z_path(:, end);
    fprintf('   ✅ 已加载 "有PPS" (ss_pps) 稳态数据。\n');
catch ME
    error('加载有PPS数据失败: %s', ME.message);
end

fprintf('   数据加载完成。\n');

%% --- 2. 生成国民经济核算报告 ---
fprintf('\n--- 2. 生成国民经济核算报告 ---\n');

% --- 生成报告结构体 (关闭命令行打印) ---
% 注意: 最后一个参数 is_bgp_ss 对 ss_nopps 是 false，对 ss_pps 是 true
report_nopps = SS.display_national_accounts(ss_nopps, cS_nopps, paramS_nopps, Z_norm_nopps, '', false, false);
report_pps = SS.display_national_accounts(ss_pps, cS_pps, paramS_pps, Z_norm_pps, '', false, true);

fprintf('   ✅ 核算报告已在内存中生成。\n');


%% --- 3. [核心] 定义比较变量并计算差异 ---
fprintf('\n--- 3. 定义比较变量并计算表格内容 ---\n');

% --- 定义要比较的变量和格式 (精简版) ---
% 格式: { 'LaTeX标签', '变量名', '格式', '变化类型', '单位' }
vars_to_compare = {
    '\multicolumn{4}{l}{\textbf{A. 宏观总量与价格}}', '', '', '', '';
    '有效人均产出 ($y$)', 'Y', '%.4f', 'abs', '';
    '有效人均私人资本 ($k_p$)', 'K_p', '%.4f', 'abs', '';
    '\quad 其中: 常规资本 ($k_k$)', 'K_k', '%.4f', 'abs', '';
    '\quad 其中: PPS 资本 ($k_{pps}$)', 'K_pps', '%.4f', 'abs', '';
    '私人资本/产出比 ($k_p/y$)', 'Kp_Y_ratio', '%.3f', 'diff', '';
    '年化真实利率 ($r$)', 'r_annual', '%.2f', 'diff', '\%';
    '有效工资率 ($w$)', 'w', '%.4f', 'abs', '';
    '\multicolumn{4}{l}{\textbf{B. 财政与养老金 (占产出Y的比例)}}', '', '', '', '';
    'PAYG缴费', 'PAYG_contrib', '%.2f', 'ratio_y', '\%';
    'PAYG福利', 'PAYG_benefit', '%.2f', 'ratio_y', '\%';
    'PPS缴费', 'PPS_contrib', '%.2f', 'ratio_y', '\%';
    'PPS提取', 'PPS_withdrawal', '%.2f', 'ratio_y', '\%';
};

% --- 初始化存储结果的 cell 数组 ---
table_data = cell(size(vars_to_compare, 1), 4); % {Label, Val_NoPPS, Val_PPS, Change}

% --- 循环处理每个变量 ---
for i = 1:size(vars_to_compare, 1)
    label = vars_to_compare{i, 1};
    var_name = vars_to_compare{i, 2};
    
    % 如果是子标题行
    if isempty(var_name)
        table_data{i, 1} = label;
        continue;
    end
    
    table_data{i, 1} = label;
    
    format_spec = vars_to_compare{i, 3};
    change_type = vars_to_compare{i, 4};
    unit = vars_to_compare{i, 5};
    
    % --- 提取数值 (增加稳健性检查) ---
    if isfield(report_nopps, var_name), val_nopps = report_nopps.(var_name); else, val_nopps = NaN; end
    if isfield(report_pps, var_name), val_pps = report_pps.(var_name); else, val_pps = NaN; end
    
    % --- 根据 ratio_y 类型预处理 ---
    if strcmp(change_type, 'ratio_y')
        if isfield(report_nopps, 'Y') && report_nopps.Y ~= 0, val_nopps = (val_nopps / report_nopps.Y) * 100; end
        if isfield(report_pps, 'Y') && report_pps.Y ~= 0, val_pps = (val_pps / report_pps.Y) * 100; end
    end
    
    % --- 格式化数值列 ---
    if isnan(val_nopps), table_data{i, 2} = '-'; else, table_data{i, 2} = [sprintf(format_spec, val_nopps), unit]; end
    if isnan(val_pps), table_data{i, 3} = '-'; else, table_data{i, 3} = [sprintf(format_spec, val_pps), unit]; end

    % --- 计算并格式化变化列 ---
    change_val_str = '-';
    if ~isnan(val_nopps) && ~isnan(val_pps)
        if strcmp(change_type, 'abs')
            if abs(val_nopps) > 1e-9, change = (val_pps / val_nopps - 1) * 100; else, change = NaN; end
            if ~isnan(change), change_val_str = sprintf('%.2f\\%%', change); end
        elseif strcmp(change_type, 'diff') || strcmp(change_type, 'ratio_y')
            change = val_pps - val_nopps;
            change_val_str = sprintf('%.2f pp', change);
        end
    end
    table_data{i, 4} = change_val_str;
end
fprintf('   ✅ 表格内容计算完成。\n');

%% --- 4. 生成 LaTeX 表格 ---
fprintf('\n--- 4. 生成 LaTeX 格式的稳态比较表格 ---\n');

% --- 准备输出目录和文件 ---
[output_dir, ~, ~] = fileparts(output_tex_filename);
if ~exist(output_dir, 'dir'), mkdir(output_dir); end
fileID = fopen(output_tex_filename, 'w', 'n', 'UTF-8');
if fileID == -1, error('无法创建或打开LaTex输出文件。'); end

% --- 写入表格框架 ---
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '%%  此文件由 ss_comparison.m (v1.0) 自动生成\n');
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '\\begin{table}[htbp]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{稳态宏观经济结果比较}\n');
fprintf(fileID, '\\label{tab:ss_comparison}\n');
fprintf(fileID, '\\begin{threeparttable}\n');
fprintf(fileID, '\\begin{tabular}{lccc}\n');
fprintf(fileID, '\\toprule\n');
fprintf(fileID, '变量 & 期初稳态 & 期末稳态 & 变化 \\\\\n');
fprintf(fileID, '\\midrule\n');

% --- 循环写入表格内容 ---
for i = 1:size(table_data, 1)
    if isempty(vars_to_compare{i, 2}) % 如果是子标题
        fprintf(fileID, '%s \\\\\n', table_data{i, 1});
    else
        row_str = strjoin(table_data(i,:), ' & ');
        fprintf(fileID, '%s \\\\\n', row_str);
    end
end

% --- 写入表格结尾和注释 ---
fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}\n');
fprintf(fileID, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID, '  \\footnotesize\n');
fprintf(fileID, '  \\item[注] “无PPS情景”对应模型初始稳态 (ss0)，“有PPS情景”对应模型终期稳态 (ssF)。\n');
fprintf(fileID, '  除特别注明外，所有流量和存量均为单位有效劳动的标准化值。财政与养老金流量已表示为占各自稳态产出(Y)的百分比。\n');
fprintf(fileID, '  “变化”列中，“\\%%”表示相对变化率，“pp”表示百分点差异。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}\n');

% --- 关闭文件 ---
fclose(fileID);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_filename);
fprintf('\n--- 稳态比较脚本执行完毕 ---\n');

# 第4节

## 图 4.1--人口冲击压力下的宏观经济转轨动态 ($\bar{g}=2\%$)

In [ ]:
% =========================================================================
% == SCRIPT: plot_figure_4_1_aging_impact_dashboard_v4.2.m
% ==
% == 版本: [v4.2 - 1x4核心变量最终版]
% ==
% == 目的:
% ==   - [核心] 最终精简版，聚焦于4个最重要的宏观变量：人均资本(k_p),
% ==     人均产出(y), 年化利率(r), 和有效工资率(w_hat)。
% ==   - [布局] 采用1x4的单行布局，使图表更紧凑，适合在论文中横向放置。
% ==   - [美学] 保持所有精修指令：移除轴标签、信息融入标题、单一图例、
% ==     固定Y轴、调整图窗尺寸、无总标题。
% ==   - [功能] 仍然保留 year_max 参数，灵活控制图表展示的时间范围。
% ==
% == 前置条件:
% ==   - TRANS/TPI_results_main_nopps.mat
% =========================================================================

clear; close all;

fprintf('--- 启动图4.1绘制脚本 (v4.2 - 1x4核心变量最终版) ---\n');

%% --- 1. 用户设定 ---
year_max = 2300; % 控制图表展示的结束年份

%% --- 2. 定义文件路径与检查 ---
file_nopps = 'TRANS/TPI_results_main_nopps.mat';
output_dir = 'tex/fig';
output_filename = fullfile(output_dir, 'fig4.1_aging_impact_dashboard.png');

if ~exist(file_nopps, 'file'), error('找不到PAYG-only结果文件: %s', file_nopps); end
if ~exist(output_dir, 'dir'), mkdir(output_dir); fprintf('   已创建输出目录: %s\n', output_dir); end

%% --- 3. [核心] 数据加载与重构 ---
fprintf('   正在加载并重构 "PAYG-only" 情景数据...\n');

% --- 数据加载 ---
data = load(file_nopps);
cS = data.cS;

% --- 路径重构 ---
total_pop_path = sum(cS.Z_path_raw, 1);
AZ_path = cS.A_path .* total_pop_path;

paths = struct();
paths.k_p = data.results.K_p_path ./ AZ_path;
paths.y   = data.results.Y_path   ./ AZ_path;
paths.r   = data.results.r_path; 
paths.w_hat = data.results.w_path ./ cS.A_path;

% 计算退休人口占比
mass_retirees_path = sum(cS.Z_path_raw((cS.aR_new+1):end, :), 1);
paths.retiree_share = mass_retirees_path ./ total_pop_path;

% --- 时间轴截取 ---
T = cS.T_sim;
time_axis_full = (cS.start_year : cS.time_Step : (cS.start_year + (T-1)*cS.time_Step));

% [!!! 核心修正: 重新定义 xlabel_text !!!]
xlabel_text = '年份'; % 确保该变量被定义

time_mask = time_axis_full <= year_max;
time_axis = time_axis_full(time_mask);

% 对所有路径应用mask
fields = fieldnames(paths);
for i = 1:length(fields)
    if size(paths.(fields{i}), 2) == T
        paths.(fields{i}) = paths.(fields{i})(time_mask);
    end
end

fprintf('   ✅ 数据加载和重构完毕。\n');

%% --- 4. 绘图 ---
fprintf('   正在生成图表...\n');
fig = figure('Name', '老龄化冲击下的宏观经济动态', 'Position', [461 603 730 319]); % 调整高度以适应1x2布局
tcl = tiledlayout(1, 2, 'Padding', 'compact', 'TileSpacing', 'normal');

% =========================================================================
% ==                      [核心美学修正 - 1x2 复合版]
% =========================================================================
% --- 1. 定义灰度图风格 ---
style_kp   = {'k-', 'LineWidth', 2.5};     % 资本: 黑色实线
style_y    = {'--', 'Color', [0.4 0.4 0.4], 'LineWidth', 2.5}; % 产出: 深灰虚线
style_w    = {':', 'Color', [0.2 0.2 0.2], 'LineWidth', 2.5}; % 工资: 黑色点线
style_r    = {'k-', 'LineWidth', 2.5};     % 利率: 黑色实线 (在自己的图里)
style_bg_line  = {'-.', 'Color', [0.8 0.8 0.8], 'LineWidth', 2}; % 背景线: 浅灰色虚线

color_left_axis = 'black';
color_right_axis = [0.5 0.5 0.5]; % 右轴: 中灰色

font_size_title = 14;
font_size_axis  = 11;
yy_lim_bar = [15 60]; % 固定Y轴范围 (百分比)

% --- 2. 准备X轴刻度 (确保包含首尾年份) ---
tick_start_year = ceil(time_axis(1)/50)*50;
xticks_vec = tick_start_year :30 : time_axis(end);
% 确保第一个和最后一个年份刻度被包括
xticks_vec = unique([time_axis(1), xticks_vec]);


% --- 面板 1: 资本、产出与工资 ---
ax1 = nexttile;
hold(ax1, 'on');

% --- 设置坐标轴本身的字体为 Times New Roman ---
% 这会影响到刻度标签（数字和英文）
set(ax1, 'FontName', 'Times New Roman');
set(ax1, 'FontSize', font_size_axis); % 保持轴字体大小


% -- 右轴: 背景线 --
yyaxis(ax1, 'right');
p_bg_line = plot(ax1, time_axis, paths.retiree_share * 100, style_bg_line{:},'DisplayName',['退休人口占比' newline '(右轴, %)']);
set(ax1, 'YColor', color_right_axis);
ylim(ax1, yy_lim_bar);
% 由于已经设置了ax1的FontName，Y轴刻度会自动继承Times New Roman。
% 如果有YLabel，显式设置其字体为SimSun
ylabel_right_ax1 = get(ax1, 'YLabel');
if ~isempty(ylabel_right_ax1)
    set(ylabel_right_ax1, 'FontName', 'SimSun');
end


% -- 左轴: 三条主曲线 --
yyaxis(ax1, 'left');
p_kp = plot(ax1, time_axis, paths.k_p, style_kp{:}, 'DisplayName', '有效人均私人资本');
p_y  = plot(ax1, time_axis, paths.y,   style_y{:},  'DisplayName', '有效人均产出');
p_w  = plot(ax1, time_axis, paths.w_hat, style_w{:},'DisplayName', '有效工资率');
set(ax1, 'YColor', color_left_axis);
ylim([0,0.8]);
% 由于已经设置了ax1的FontName，Y轴刻度会自动继承Times New Roman。
% 如果有YLabel，显式设置其字体为SimSun
ylabel_left_ax1 = get(ax1, 'YLabel');
if ~isempty(ylabel_left_ax1)
    set(ylabel_left_ax1, 'FontName', 'SimSun');
end


% -- 图例和标签 --
% 标题字体设为SimSun
title_obj_ax1 = title(ax1, '(a) 资本、产出与工资', 'FontSize', font_size_title);
set(title_obj_ax1, 'FontName', 'SimSun');



% 图例：为了确保中文显示，将图例的字体设置为SimSun
legend_obj_ax1 = legend(ax1, [p_kp, p_y, p_w, p_bg_line], 'Location', 'southeast', 'FontSize', 10, 'Box', 'off');
set(legend_obj_ax1, 'FontName', 'SimSun');


% -- 通用设置 --
grid(ax1, 'on');
xlim(ax1, [time_axis(1), time_axis(end)]);
set(ax1, 'XTick', xticks_vec);
xtickangle(ax1, 40);
% X轴刻度（数字和字母）的字体已通过set(ax1, 'FontName', 'Times New Roman')设置。

hold(ax1, 'off');

% --- 面板 2: 年化真实利率 ---
ax2 = nexttile;
hold(ax2, 'on');

% --- 设置坐标轴本身的字体为 Times New Roman ---
set(ax2, 'FontName', 'Times New Roman');
set(ax2, 'FontSize', font_size_axis); % 保持轴字体大小


% -- 右轴: 背景线 --
yyaxis(ax2, 'right');
p_bg_line = plot(ax2, time_axis, paths.retiree_share * 100, style_bg_line{:});
set(ax2, 'YColor', color_right_axis);
ylim(ax2, yy_lim_bar);



% -- 左轴: 利率曲线 --
yyaxis(ax2, 'left');
r_annual = (1 + paths.r).^(1/cS.time_Step) - 1;
plot(ax2, time_axis, r_annual * 100, style_r{:});
set(ax2, 'YColor', color_left_axis);
ylim(ax2, 'padded');


% 图例：为了确保中文显示，将图例的字体设置为SimSun
legend_obj_ax2 = legend(ax2, p_bg_line, ['退休人口占比' newline '(右轴, %)'], ...
    'Location', 'east', 'FontSize', 10, 'Box', 'off', 'FontName', 'SimSun');


% -- 标签 --
% 标题字体设为SimSun
title_obj_ax2 = title(ax2, '(b) 年化真实利率 (%)', 'FontSize', font_size_title, 'FontName', 'SimSun');


% -- 通用设置 --
grid(ax2, 'on');
xlim(ax2, [time_axis(1), time_axis(end)]);
set(ax2, 'XTick', xticks_vec);
xtickangle(ax2, 40);
% X轴刻度（数字和字母）的字体已通过set(ax2, 'FontName', 'Times New Roman')设置。

hold(ax2, 'off');
%% --- 5. 保存图像 ---
fprintf('   正在保存图像至: %s\n', output_filename);
try
    exportgraphics(fig, output_filename, 'Resolution', 600);
    fprintf('   ✅ 图像保存成功。\n');
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end

fprintf('\n--- 脚本执行完毕 ---\n');

## 图 4.2--固定替代率承诺下的社保基金动态与财政压力 ($\bar{\rho}=40\%$)

In [ ]:
% =========================================================================
% == SCRIPT: exo_pension.m
% ==
% == 版本: [v2.1 - 1x2灰度精修版]
% ==
% == 目的:
% ==   - 采用1x2的横向布局，生成一幅紧凑、专业的灰度图。
% ==   - (a) 左图展示基金存量动态。
% ==   - (b) 右图展示PAYG收支流量动态。
% ==   - 严格遵循特定的尺寸、字体、刻度和颜色要求。
% ==
% == 前置条件:
% ==   - 'TRANS/TPI_results_main_nopps.mat'
% ==   - 'TRANS/iter_results_main_nopps.mat'
% =========================================================================

clear; close all;
addpath(pwd);

fprintf('--- 启动外生养老金基金模拟与绘图脚本 (v2.1) ---\n');

%% --- 1. 用户设定 ---
exo_rho = 0.40;
initial_fund_to_gdp_ratio = 0.055;
year_max = 2100;

%% --- 2. 定义文件路径与加载数据 ---
file_tpi_results = 'TRANS/TPI_results_main_nopps.mat';
file_iter_results = 'TRANS/iter_results_main_nopps.mat';
output_dir = 'tex/fig';
output_filename = fullfile(output_dir, 'fig4.2_exo_pension_fund.png');

if ~exist(file_tpi_results, 'file'), error('找不到TPI结果文件: %s', file_tpi_results); end
if ~exist(file_iter_results, 'file'), error('找不到TPI迭代文件: %s', file_iter_results); end
if ~exist(output_dir, 'dir'), mkdir(output_dir); fprintf('   已创建输出目录: %s\n', output_dir); end

fprintf('   正在加载基准路径数据...\n');
data_tpi = load(file_tpi_results);
data_iter = load(file_iter_results);

cS = data_tpi.cS;
results = data_tpi.results;
b_hat_path_iter = data_iter.b_hat_path_iter;
T = cS.T_sim;

fprintf('   ✅ 数据加载完毕。\n');

%% --- 3. [核心] 会计核算 ---
fprintf('   正在进行养老金缺口与基金动态的会计核算...\n');

r_path_level = results.r_path;
Y_path_level = results.Y_path;
w_hat_path = results.w_path ./ cS.A_path;
mass_retirees_path = sum(cS.Z_path_raw((cS.aR_new+1):end, :), 1);

b_hat_path_exo = exo_rho * w_hat_path;
b_hat_delta = b_hat_path_exo - b_hat_path_iter;

P_level_path = zeros(1, T);
P_level_path(1) = initial_fund_to_gdp_ratio * Y_path_level(1);
fprintf('   初始(t=1)基金余额设定为: %.4f (基于GDP的%.2f%%)\n', P_level_path(1), initial_fund_to_gdp_ratio*100);

for t = 1:(T-1)
    B_delta_total_level = b_hat_delta(t) * cS.A_path(t) * mass_retirees_path(t);
    P_level_path(t+1) = P_level_path(t) * (1 + r_path_level(t)) - B_delta_total_level;
end

Total_PAYG_Contrib_level = b_hat_path_iter .* cS.A_path .* mass_retirees_path;
Total_Promised_Benefit_level = b_hat_path_exo .* cS.A_path .* mass_retirees_path;

contrib_gdp_ratio = Total_PAYG_Contrib_level ./ Y_path_level;
benefit_gdp_ratio = Total_Promised_Benefit_level ./ Y_path_level;

fprintf('   ✅ 会计核算完成。\n');

%% --- 4. 准备绘图数据 ---
fprintf('   正在准备绘图数据...\n');
paths = struct();
paths.fund_level = P_level_path;
paths.retiree_share = mass_retirees_path ./ sum(cS.Z_path_raw, 1);
paths.contrib_gdp = contrib_gdp_ratio;
paths.benefit_gdp = benefit_gdp_ratio;

time_axis_full = (cS.start_year : cS.time_Step : (cS.start_year + (T-1)*cS.time_Step));
time_mask = time_axis_full <= year_max;
time_axis = time_axis_full(time_mask);

fields = fieldnames(paths);
for i = 1:length(fields)
    if size(paths.(fields{i}), 2) == T
        paths.(fields{i}) = paths.(fields{i})(time_mask);
    end
end
fprintf('   图表将展示到 %d 年。\n', year_max);

%% --- 5. [核心] 绘图 ---
fprintf('   正在生成图表...\n');
fig = figure('Name', '外生社保基金动态与流量分析', 'Position', [461, 603, 730, 319]);
tcl = tiledlayout(1, 2, 'Padding', 'compact', 'TileSpacing', 'compact');

% --- 统一定义灰度风格和格式 ---
font_size_title = 12;
font_size_axis  = 10;
color_axis_main = 'black';
color_axis_bg = [0.5 0.5 0.5];



% ================= PANEL (a): 基金存量动态 =================
ax1 = nexttile;
hold(ax1, 'on');

style_fund = {'k-', 'LineWidth', 2};
style_retiree_share = {'--', 'Color', [0.5 0.5 0.5], 'LineWidth', 2};
style_zero_line = {'--', 'Color', [0.7 0.7 0.7], 'LineWidth', 1};

set(ax1, 'FontName', 'Times New Roman', 'FontSize', font_size_axis);

yyaxis(ax1, 'right');
p_retiree = plot(ax1, time_axis, paths.retiree_share * 100, style_retiree_share{:}, 'DisplayName', '退休人口占比 (右轴, %)');
set(ax1, 'YColor', color_axis_bg, 'YLim', [15 70]);

yyaxis(ax1, 'left');
p_fund = plot(ax1, time_axis, paths.fund_level, style_fund{:}, 'DisplayName', '社保基金余额 (左轴)');
ylim([min(paths.fund_level)*0.9 max(paths.fund_level)*2])
yline(ax1, 0, style_zero_line{:});
set(ax1, 'YColor', color_axis_main);
ylabel(ax1, '基金余额 (万亿元)', 'FontName', 'SimSun');


title_obj1 = title(ax1, '(a) 基金存量动态', 'FontSize', font_size_title, 'FontName', 'SimSun');
legend_obj1 = legend(ax1, [p_fund, p_retiree], 'Location', 'south', 'FontSize', 9, 'Box', 'off', 'FontName', 'SimSun');

grid(ax1, 'on'); box(ax1, 'on');
xlim(ax1, [time_axis(1), time_axis(end)]);
set(ax1, 'XTick', time_axis);
xtickangle(ax1, 40);
hold(ax1, 'off');

% ================= PANEL (b): PAYG收支流量 =================
ax2 = nexttile;
hold(ax2, 'on');

style_contrib = {'k-', 'LineWidth', 2};
style_benefit = {'--', 'Color', [0.4 0.4 0.4], 'LineWidth', 2};
color_deficit_area = [0.8 0.8 0.8]; % 浅灰色区域

set(ax2, 'FontName', 'Times New Roman', 'FontSize', font_size_axis);

deficit_gdp = paths.benefit_gdp - paths.contrib_gdp;
p_area = area(ax2, time_axis, deficit_gdp * 100, ...
    'FaceColor', color_deficit_area, 'FaceAlpha', 0.6, ...
    'EdgeColor', 'none', 'DisplayName', 'PAYG缺口');

p_contrib = plot(ax2, time_axis, paths.contrib_gdp * 100, style_contrib{:}, 'DisplayName', 'PAYG缴费收入');
p_benefit = plot(ax2, time_axis, paths.benefit_gdp * 100, style_benefit{:}, 'DisplayName', '承诺的福利支出');

ylim(ax2, [0, max(paths.benefit_gdp)*100*1.1]);
ylabel(ax2, '占GDP的百分比 (%)', 'FontName', 'SimSun');

title_obj2 = title(ax2, '(b) PAYG收支流量', 'FontSize', font_size_title, 'FontName', 'SimSun');
legend_obj2 = legend(ax2, [p_contrib, p_benefit, p_area], 'Location', 'northwest', 'FontSize', 9, 'Box', 'off', 'FontName', 'SimSun');

grid(ax2, 'on'); box(ax2, 'on');
xlim(ax2, [time_axis(1), time_axis(end)]);
set(ax2, 'XTick', time_axis);
xtickangle(ax2, 40);
hold(ax2, 'off');


fprintf('   ✅ 图表生成完毕。\n');

%% --- 6. 保存图像 ---
fprintf('   正在保存图像至: %s\n', output_filename);
try
    exportgraphics(fig, output_filename, 'Resolution', 600);
    fprintf('   ✅ 图像保存成功。\n');
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end

fprintf('\n--- 脚本执行完毕 ---\n');

## 表 4.1--引入个人养老金制度对不同年龄组福利的影响 (截面分析)

In [ ]:
% =========================================================================
% == SCRIPT: welfare_analysis.m
% == 版本: [v2.1 - 价值函数重构简化版]
% ==
% == 目的:
% ==   - [核心修正] 简化了价值函数重构过程，移除了对 `final_aggr_supply`
% ==     和 `calculate_endogenous_TR_path` 的依赖，解决了 "无法识别的
% ==     字段名称" 错误。
% ==   - 现在直接使用 TPI 迭代收敛后的宏观价格和福利路径作为反向求解的输入。
% ==   - 分析逻辑和最终输出的表格结构保持不变。
% ==
% == 前置条件:
% ==   - 已运行 `fix_tpi_results_variables.m` 修正了数据文件。
% ==   - TRANS/TPI_results_main_nopps.mat
% ==   - TRANS/TPI_results_main_pps.mat
% ==   - SS/data_for_transition_nopps.mat
% ==   - SS/data_for_transition_pps.mat
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== OLG模型年龄组截面福利分析脚本 (v2.1) ===\n\n');

%% --- 1. 设置与加载数据 ---
fprintf('--- 1. 设置与加载数据 ---\n');

% --- 文件路径定义 ---
file_path_nopps = 'TRANS/TPI_results_main_nopps.mat';
file_path_pps = 'TRANS/TPI_results_main_pps.mat';
iter_file_nopps = 'TRANS/iter_results_main_nopps.mat';
iter_file_pps = 'TRANS/iter_results_main_pps.mat';
ss_data_nopps_path = 'SS/data_for_transition_nopps.mat';
ss_data_pps_path = 'SS/data_for_transition_pps.mat';
output_tex_filename = 'tex/tab/tab4.1_welfare_analysis_by_age_group.tex';

% --- 加载数据 ---
fprintf('   正在加载 "无PPS" (基准) 和 "有PPS" (改革) 情景数据...\n');
data_nopps = load(file_path_nopps);
iter_data_nopps = load(iter_file_nopps);
ss_data_nopps = load(ss_data_nopps_path);

data_pps = load(file_path_pps);
iter_data_pps = load(iter_file_pps);
ss_data_pps = load(ss_data_pps_path);
fprintf('   ✅ 数据加载完成。\n');


%% --- 2. [核心修正] 简化价值函数路径重构 ---
fprintf('\n--- 2. 重构价值函数路径 (简化版) ---\n');

% --- a. 重构 "无PPS" 情景的价值函数路径 ---
fprintf('   正在为 "无PPS" 情景运行反向迭代...\n');
cS_nopps = data_nopps.cS;
paramSF_nopps = ss_data_nopps.data_for_transition.paramSF;
pathS_nopps = struct();
pathS_nopps.r_path = data_nopps.results.r_path;
pathS_nopps.w_hat_path = data_nopps.results.w_path ./ cS_nopps.A_path;
% 直接使用 TPI 迭代收敛的路径作为输入
pathS_nopps.b_hat_path = iter_data_nopps.b_hat_path_iter;
pathS_nopps.tr_per_hh_hat_path = iter_data_nopps.TR_hat_pc_path_iter;
% 填充外生路径
A_path_ext_nopps = [cS_nopps.A_path, cS_nopps.A_path(end) * (1 + ((1+cS_nopps.g_A_ss)^cS_nopps.time_Step-1))];
pathS_nopps.g_A_path = A_path_ext_nopps(2:end)./A_path_ext_nopps(1:end-1)-1;
pathS_nopps.theta_path = cS_nopps.theta_path;
% 运行反向迭代
[~, Val_path_nopps] = household.backward_hh(pathS_nopps, cS_nopps, paramSF_nopps, ...
    ss_data_nopps.data_for_transition.valF, ss_data_nopps.data_for_transition.polF);

% --- b. 重构 "有PPS" 情景的价值函数路径 ---
fprintf('   正在为 "有PPS" 情景运行反向迭代...\n');
cS_pps = data_pps.cS;
paramSF_pps = ss_data_pps.data_for_transition.paramSF;
pathS_pps = struct();
pathS_pps.r_path = data_pps.results.r_path;
pathS_pps.w_hat_path = data_pps.results.w_path ./ cS_pps.A_path;
% 直接使用 TPI 迭代收敛的路径作为输入
pathS_pps.b_hat_path = iter_data_pps.b_hat_path_iter;
pathS_pps.tr_per_hh_hat_path = iter_data_pps.TR_hat_pc_path_iter;
% 填充外生路径
A_path_ext_pps = [cS_pps.A_path, cS_pps.A_path(end) * (1 + ((1+cS_pps.g_A_ss)^cS_pps.time_Step-1))];
pathS_pps.g_A_path = A_path_ext_pps(2:end)./A_path_ext_pps(1:end-1)-1;
pathS_pps.theta_path = cS_pps.theta_path;
% 运行反向迭代
[~, Val_path_pps] = household.backward_hh(pathS_pps, cS_pps, paramSF_pps, ...
    ss_data_pps.data_for_transition.valF, ss_data_pps.data_for_transition.polF);

fprintf('   ✅ 价值函数路径重构完成。\n');


%% --- 3. [核心] 计算特定截面下各年龄组的福利变化 (CV) ---
fprintf('\n--- 3. 计算各年龄组在特定年份的福利变化 ---\n');

% --- 定义分析维度 ---
assessment_years = [2023, 2030,2040, 2050,2070,2100,2200]; % 选择要进行截面分析的年份

% --- 定义代表性年龄组 (基于模型年龄索引 a_idx) ---
% 假设 cS.age1_orig = 20, cS.time_Step = 5
% a=1 -> 20-24岁, a=2 -> 25-29岁, etc.
aR_idx = cS_nopps.aR_new; % 退休年龄索引
working_age_groups = {
    '青年 (25-29岁)', round((25 - cS_nopps.age1_orig)/cS_nopps.time_Step) + 1;
    '中年 (45-49岁)', round((45 - cS_nopps.age1_orig)/cS_nopps.time_Step) + 1;
    '临近退休 (60-64岁)', aR_idx
};
retired_age_groups = {
    '刚退休 (65-69岁)', aR_idx + 1;
    '中年退休 (75-79岁)', aR_idx + 3;
    '高龄退休 (85-89岁)', aR_idx + 5
};
age_groups_to_analyze = [working_age_groups; retired_age_groups];
num_age_groups = size(age_groups_to_analyze, 1);

% --- 初始化结果存储 ---
% 使用 table 来存储，更易于阅读和导出
varTypes = ['string', repmat("string", 1, length(assessment_years))];
varNames = ['age_group', arrayfun(@(y) sprintf('年份_%d', y), assessment_years, 'UniformOutput', false)];
results_table = table('Size', [num_age_groups, length(varNames)], 'VariableTypes', varTypes, 'VariableNames', varNames);
results_table.age_group = age_groups_to_analyze(:,1);

% --- 核心计算循环 ---
sigma = cS_nopps.sigma;

for col_idx = 1:length(assessment_years)
    year = assessment_years(col_idx);
    t_idx = round((year - cS_nopps.start_year) / cS_nopps.time_Step) + 1;
    
    fprintf('   正在评估年份: %d (t=%d)...\n', year, t_idx);
    
    if t_idx > cS_nopps.T_sim
        fprintf('      警告: 评估年份超出模拟范围，此列将留空。\n');
        continue;
    end
    
    for row_idx = 1:num_age_groups
        age_label = age_groups_to_analyze{row_idx, 1};
        a_idx = age_groups_to_analyze{row_idx, 2};
        
        if a_idx > cS_nopps.aD_new
            results_table(row_idx, col_idx+1) = {"-"};
            continue;
        end
        
        % --- 计算期望剩余效用 (无PPS) ---
        dist_nopps = data_nopps.final_Dist_path(:, :, :, a_idx, t_idx);
        val_nopps = Val_path_nopps(:, :, :, a_idx, t_idx);
        mass_nopps = sum(dist_nopps, 'all');
        EV_nopps = sum(val_nopps .* dist_nopps, 'all') / max(1e-9, mass_nopps);
        
        % --- 计算期望剩余效用 (有PPS) ---
        dist_pps = data_pps.final_Dist_path(:, :, :, a_idx, t_idx);
        val_pps = Val_path_pps(:, :, :, a_idx, t_idx);
        mass_pps = sum(dist_pps, 'all');
        EV_pps = sum(val_pps .* dist_pps, 'all') / max(1e-9, mass_pps);
        
        % --- 计算 CV ---
        if EV_nopps >= 0 || EV_pps >= 0
            cv_str = 'N/A';
        else
            cv = (EV_pps / EV_nopps)^(1 / (1 - sigma)) - 1;
            cv_str = sprintf('%.2f\\%%', cv * 100);
        end
        results_table(row_idx, col_idx+1) = {cv_str};
    end
end
disp('福利分析结果:');
disp(results_table);
fprintf('   ✅ 所有年龄组福利计算完成。\n');

%% --- 4. 生成 LaTeX 表格 ---
fprintf('\n--- 4. 生成 LaTeX 格式的福利分析表格 ---\n');

% --- 准备输出目录和文件 ---
[output_dir, ~, ~] = fileparts(output_tex_filename);
if ~exist(output_dir, 'dir'), mkdir(output_dir); end
fileID = fopen(output_tex_filename, 'w', 'n', 'UTF-8');
if fileID == -1, error('无法创建或打开LaTex输出文件。'); end

% --- 写入表格头部 ---
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '%%  此文件由 welfare_analysis.m (v2.3) 自动生成\n');
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '\\begin{table}[htbp]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{引入个人养老金制度对不同年龄组福利的影响 (截面分析)}\n');
fprintf(fileID, '\\label{tab4.1_welfare_analysis_by_age_group}\n');
fprintf(fileID, '\\begin{threeparttable}\n');

% --- 动态生成 tabular 列定义 ---
num_year_cols = length(assessment_years);
col_defs = ['l', repmat('c', 1, num_year_cols)];
fprintf(fileID, '\\begin{tabular}{%s}\n', col_defs);
fprintf(fileID, '\\toprule\n');

% --- [!!! 核心修正: 创建两级表头 !!!] ---
% --- 第一行表头: 使用 multicolumn ---
%  - 第一个单元格留空
%  - 第二个单元格横跨所有年份列
fprintf(fileID, ' & \\multicolumn{%d}{c}{评估年份} \\\\\n', num_year_cols);
%  - 在两行表头之间增加一条细线以改善视觉效果
fprintf(fileID, '\\cmidrule(lr){2-%d}\n', num_year_cols + 1);

% --- 第二行表头: 具体的年份 ---
year_header_cells = cellfun(@(y) num2str(y), num2cell(assessment_years), 'UniformOutput', false);
year_header_line = strjoin(year_header_cells, ' & ');
fprintf(fileID, '年龄组 & %s \\\\\n', year_header_line);
fprintf(fileID, '\\midrule\n');


% --- 插入工作期和退休期子标题 ---
fprintf(fileID, '\\multicolumn{%d}{l}{\\textit{工作期}} \\\\\n', num_year_cols + 1);

% --- 循环写入工作期数据 ---
for i = 1:size(working_age_groups, 1)
    temp_row_cell = cell(1, width(results_table));
    for j = 1:width(results_table)
        value = results_table{i, j};
        temp_row_cell{j} = convert_to_string(value);
    end
    row_data_str = strjoin(temp_row_cell, ' & ');
    fprintf(fileID, '%s \\\\\n', row_data_str);
end

fprintf(fileID, '\\midrule\n');
fprintf(fileID, '\\multicolumn{%d}{l}{\\textit{退休期}} \\\\\n', num_year_cols + 1);

% --- 循环写入退休期数据 ---
for i = 1:size(retired_age_groups, 1)
    row_idx = size(working_age_groups, 1) + i;
    temp_row_cell = cell(1, width(results_table));
    for j = 1:width(results_table)
        value = results_table{row_idx, j};
        temp_row_cell{j} = convert_to_string(value);
    end
    row_data_str = strjoin(temp_row_cell, ' & ');
    fprintf(fileID, '%s \\\\\n', row_data_str);
end

% --- 写入表格结尾和注释 ---
fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}\n');
fprintf(fileID, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID, '  \\footnotesize\n');
fprintf(fileID, '  \\item[注] 表中数值为“有PPS情景”相对于“无PPS情景”的补偿变化(CV)。\n');
fprintf(fileID, '  CV值为正，代表该年龄组在评估年份的剩余生命周期福利因引入PPS制度而提高。例如，+1.00\\%%表示该群体愿意放弃其剩余生命周期消费的1\\%%，以换取从“无PPS”制度切换到“有PPS”制度。计算基于各年龄组在该时刻的期望剩余终身效用。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}\n');

% --- 关闭文件 ---
fclose(fileID);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_filename);
fprintf('\n--- 福利分析脚本执行完毕 ---\n');


% --- [辅助函数] ---
function str_out = convert_to_string(value_in)
    % 一个稳健的函数，将不同类型的数据转换为字符串
    if isstring(value_in)
        str_out = char(value_in);
    elseif ischar(value_in)
        str_out = value_in;
    elseif isnumeric(value_in) || islogical(value_in)
        str_out = num2str(value_in);
    elseif iscell(value_in) % 如果输入是cell，递归处理
        str_out = convert_to_string(value_in{1});
    else
        str_out = 'Unsupported Type';
    end
end